In [1]:
import os
import socket
import subprocess
import pandas as pd
import io
from contextlib import redirect_stdout
import itertools
import warnings
from datetime import datetime
import time
import timeout_decorator
import numpy as np
import matplotlib.pyplot as plt



In [2]:
def islocal(): return socket.gethostname()=="rc-idsia-macBook.local"
def isserver(): return socket.gethostname()=="ec6120930b2d"
def strdate(): return datetime.today().strftime('%Y-%m-%d-%H-%M-%S')


In [3]:
if islocal():
    prj_path = "/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/crema/CreMA"
    os.environ['JAVA_HOME'] = "/Library/Java/JavaVirtualMachines/openjdk-12.0.1.jdk/Contents/Home"
elif isserver():
    prj_path="/home/rcabanas/work_java_crema/crema/CreMA"
else:
    warnings.warn("Unknown server, set prj_path variable manually.")
os.chdir(prj_path)
!pwd

/home/rcabanas/work_java_crema/crema/CreMA


In [4]:
exp_folder = f"{prj_path}/papers/pgm20/experiments/"
log_folder = f"{exp_folder}/logs/"
res_folder = f"{exp_folder}/results/"
jarfile=f"{prj_path}/target/CreMA-0.1.1-SNAPSHOT-jar-with-dependencies.jar"

rebuild = False

In [5]:
if rebuild:
    !mvn install:install-file -Dfile=./external/lpsolve55j.jar -DgroupId=net.sf.lpsolve -DartifactId=lp_solve -Dversion=5.5.2 -Dpackaging=jar
    !mvn install:install-file -Dfile=./external/polco.jar -DgroupId=ch.javasoft.polco -DartifactId=polco -Dversion=4.7.1 -Dpackaging=jar
    !mvn clean compile assembly:single

In [59]:
@timeout_decorator.timeout(7*6*60)
def run_java(jarfile, javafile):
    cmd=f"java -cp {jarfile} {javafile}"
    result =subprocess.run(cmd, universal_newlines = True, shell=True,stdout = subprocess.PIPE)
    print(result.stdout)
    return result

# ChainNonMarkovian 6 5 1 -1 0 CCALP 1234
def run_chain(model, N, endovarsize, exovarsize, target, obsvar, dovar, method, seed):
    
    
    print(strdate())
    exovarsize = exovarsize or endovarsize*endovarsize + 1
    
    if obsvar is None: obsvar = -1;
    elif obsvar<0: obsvar = N + obsvar;
    
    if dovar is None: dovar = -1;    
    elif dovar<0: dovar = N + dovar;
    
    if target is None: target = N//2;
    elif target<0: target = N + target;
    
    warmups = 2
    repetitions = 5
    eps = 0.0001
    #ChainNonMarkovian 4  -v 5 -V 9 -t 1 -o -1 -d 0 -m  CCALP --seed 12234 --warmpus 0 --repetitions 1
    javafile = f"{exp_folder}/RunExperiments.java {model} {N} -v {endovarsize} -V {exovarsize} -t {target} -o {obsvar} -d {dovar} -m {method} -e {eps} -s {seed} -w {warmups} -r {repetitions} "  
    try:
        result = run_java(jarfile, javafile)
        output = [float(x) for x in result.stdout.splitlines()[-1].split(",")]
    except:
        output = [float("inf"),float("inf")]+[float("nan")]* (endovarsize*2)
        
    return output

def run_chain_markovian(N=4, endovarsize=2, exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("ChainMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_chain_nonmarkovian(N=4, endovarsize=2,exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("ChainNonMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)



def run_chain_terbin_markovian(N=4, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("TerBinChainMarkovian", N, 2, None, target, obsvar, dovar, method, seed)

def run_chain_terbin_nonmarkovian(N=4, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("TerBinChainNonMarkovian", N, 2, None, target, obsvar, dovar, method, seed)



In [60]:
run_chain_terbin_nonmarkovian(N=5, target=1, method="CCVE")

2020-05-08-09-25-20

TerBinChainNonMarkovian
   N=5 endovarsize=2 exovarsize=5 target=1 obsvar=4 dovar=0 method=CCVE seed=1234
Running experiments...
Warm-up #0 in 562.514 ms.
Warm-up #1 in 145.905 ms.
Measurement #0 in 109.321 ms.
Measurement #1 in 117.76 ms.
Measurement #2 in 119.209 ms.
Measurement #3 in 101.502 ms.
Measurement #4 in 89.584 ms.
107.4752,25.031200000000002,0.12925764192139738,0.18830536004331347,0.011694639956686521,0.07074235807860263



[107.4752,
 25.031200000000002,
 0.12925764192139738,
 0.18830536004331347,
 0.011694639956686521,
 0.07074235807860263]

In [ ]:
[float("nan"), 2]

In [10]:
run_chain_markovian(N=5, dovar=0, obsvar=None, target=-1, method="CCVE")


ChainMarkovian
   N=5 exovarsize=6 target=4 obsvar=-1 dovar=0 method=CCVE seed=1234
Running experiments...
set timeout 600 s.
Warm-up #0 in 679.624 ms.
set timeout 600 s.
Warm-up #1 in 212.373 ms.
set timeout 300 s.
Measurement #0 in 202.28 ms. size=0.0
set timeout 300 s.
Measurement #1 in 188.426 ms. size=0.0
set timeout 300 s.
Measurement #2 in 149.553 ms. size=0.0
set timeout 300 s.
Measurement #3 in 150.14 ms. size=0.0
set timeout 300 s.
Measurement #4 in 145.872 ms. size=0.0
167.2542,34.712599999999995,0.0,0.62254336,0.62254336



[167.2542, 34.712599999999995, 0.0, 0.62254336, 0.62254336]

In [50]:
def run_experiments(f, args, outkeys):

    non_evaluable = []
                      
    def is_evaluable(args): 
        return not {k:v for (k,v) in args.items() if k not in ["N"]} in \
    [{k:v for (k,v) in a.items() if k not in ["N"]} for a in non_evaluable]
         
        
    result = pd.DataFrame(columns=list(args.keys())+list(outkeys))
    log_file = f"{log_folder}{strdate()}_{f.__name__}.txt"

    for v in itertools.product(*list(args.values())):
        
        argsv = dict(zip(list(args.keys()), v))
        
        strio = io.StringIO()
        with open(log_file, 'a+') as logger:
            with redirect_stdout(strio):
                if is_evaluable(argsv):
                    outvals = f(**argsv)
                    if np.isnan(outvals).any(): non_evaluable.append(argsv)
                else:
                    outvals=[float("nan")]*len(outkeys)
            logger.write(strio.getvalue())
        result = result.append({**argsv, **dict(zip(list(outkeys), outvals))}, ignore_index=True)
        print(dict(result.iloc[-1]))
    
    return result

def get_args(**kwargs): return kwargs

In [53]:
SEED = [1234+i*1234 for i in range(10)]
res = {}
#SEED = [1234+i*1234 for i in range(3)]
#SEED = [1234]

{'a': 1}

In [ ]:
# with evidence chain markovian 

endovarsize = 2
args = get_args(N=list(range(3,20,1)), dovar=[0], obsvar=[-1], target=[None, 1], seed=SEED, exovarsize=[6], method=["CVE", "CCVE","CCALPeps"])
outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])

results1 = run_experiments(run_chain_markovian, args, outkeys)
res_file = f"{res_folder}{strdate()}_chain_markovian_ev.csv"
results1.to_csv(res_file)

res["chain_markovian_ev"] = results1

{'N': 3, 'dovar': 0, 'obsvar': -1, 'target': None, 'seed': 1234, 'exovarsize': 6, 'method': 'CVE', 'time': 2.2131999999999996, 'query_time': 1.7384, 'lowerbound0': 0.1951715374841169, 'upperbound0': 0.1951715374841169, 'lowerbound1': 0.0048284625158831005, 'upperbound1': 0.0048284625158831005}
{'N': 3, 'dovar': 0, 'obsvar': -1, 'target': None, 'seed': 1234, 'exovarsize': 6, 'method': 'CCVE', 'time': 99.6492, 'query_time': 4.651, 'lowerbound0': 0.1951715374841169, 'upperbound0': 0.1951715374841169, 'lowerbound1': 0.0048284625158831005, 'upperbound1': 0.0048284625158831005}
{'N': 3, 'dovar': 0, 'obsvar': -1, 'target': None, 'seed': 1234, 'exovarsize': 6, 'method': 'CCALPeps', 'time': 378.08959999999996, 'query_time': 375.3598, 'lowerbound0': 0.19515966731328047, 'upperbound0': 0.1951596673132805, 'lowerbound1': 0.004840332686719516, 'upperbound1': 0.004840332686719526}
{'N': 3, 'dovar': 0, 'obsvar': -1, 'target': None, 'seed': 2468, 'exovarsize': 6, 'method': 'CVE', 'time': 2.3812, 'quer

In [ ]:
## with evidence chain NON markovian 
endovarsize = 2    
args = get_args(N=list(range(20,25,1)), dovar=[0], obsvar=[-1], target=[None, 1], seed=SEED, exovarsize=[6], method=["CVE", "CCALPeps"])
outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])

results1 = run_experiments(run_chain_nonmarkovian, args, outkeys)
res_file = f"{res_folder}{strdate()}_chain_nonmarkovian_ev.csv"
results1.to_csv(res_file)
                     
res["chain_nonmarkovian_ev"] = results1

{'N': 20, 'dovar': 0, 'obsvar': -1, 'target': None, 'seed': 1234, 'exovarsize': 6, 'method': 'CVE', 'time': 16927.2122, 'query_time': 16926.3796, 'lowerbound0': 0.10287412267443945, 'upperbound0': 0.10287412267443945, 'lowerbound1': 0.09712587732556056, 'upperbound1': 0.09712587732556056}
{'N': 20, 'dovar': 0, 'obsvar': -1, 'target': None, 'seed': 2468, 'exovarsize': 6, 'method': 'CVE', 'time': 17550.417599999997, 'query_time': 17549.659200000002, 'lowerbound0': 0.03379999999999999, 'upperbound0': 0.03379999999999999, 'lowerbound1': 0.16620000000000001, 'upperbound1': 0.16620000000000001}
{'N': 20, 'dovar': 0, 'obsvar': -1, 'target': None, 'seed': 2468, 'exovarsize': 6, 'method': 'CCALPeps', 'time': 6565.2538, 'query_time': 6558.7991999999995, 'lowerbound0': 0.03381447850409437, 'upperbound0': 0.03381605711459971, 'lowerbound1': 0.16618394288540028, 'upperbound1': 0.16618552149590562}
{'N': 20, 'dovar': 0, 'obsvar': -1, 'target': None, 'seed': 3702, 'exovarsize': 6, 'method': 'CVE', 't

In [63]:
res["chain_nonmarkovian_ev"]

,N,dovar,obsvar,target,seed,exovarsize,method,time,query_time,lowerbound0,upperbound0,lowerbound1,upperbound1
0,3,0,-1,None,1234,6,CVE,2.0198,1.6724,0.029001,0.029001,0.170999,0.170999
1,3,0,-1,None,1234,6,CCVE,65.7938,3.5858,0.029001,0.029001,0.170999,0.170999
2,3,0,-1,None,1234,6,CCALPeps,351.6086,349.3460,0.029028,0.029031,0.170969,0.170972
3,3,0,-1,None,2468,6,CVE,2.0726,1.6808,0.191930,0.191930,0.008070,0.008070
4,3,0,-1,None,2468,6,CCVE,64.8444,3.6546,0.191930,0.191930,0.008070,0.008070
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,19,0,-1,1,11106,6,CCVE,NaN,NaN,NaN,NaN,NaN,NaN
1016,19,0,-1,1,11106,6,CCALPeps,7525.3738,7519.2710,0.016051,0.175943,0.024057,0.183949
1017,19,0,-1,1,12340,6,CVE,11358.9546,11358.2238,0.078000,0.078000,0.122000,0.122000
1018,19,0,-1,1,12340,6,CCVE,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# without evidence chain markovian 
#args = {"N":list(range(5,15)), "dovar":[0], "target":[-1], "obsvar":[None], "seed":[1234], "method":["CVE", "CCVE", "CCALP", "CCALPeps"]}
#results3 = run_experiments(run_chain_markovian, args)
#res_file = f"{res_folder}{strdate()}_chain_markovian_nonev.csv"
#results3.to_csv(res_file)
#results3

In [ ]:
# without evidence chain non markovian
#args = {"N":list(range(5,30,2)), "dovar":[0], "target":[-1], "obsvar":[None], "seed":[1234], "method":["CVE", "CCVE", "CCALP", "CCALPeps"]}
#results4 = run_experiments(run_chain_nonmarkovian, args)
#res_file = f"{res_folder}{strdate()}_chain_nonmarkovian_nonev.csv"
#results4.to_csv(res_file)
#results4

In [ ]:
# TerBin chain markovian
args = {"N":list(range(3,20,1)),"dovar":[0], "target":[1], "obsvar":[-1],  "seed":SEED, "method":["CVE", "CCVE","CCALPeps"]}
outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,3)], [])

results1 = run_experiments(run_chain_terbin_markovian, args, outkeys)
res_file = f"{res_folder}{strdate()}_chain_terbin_markovian_ev.csv"
results1.to_csv(res_file)

results1

In [27]:
# TerBin chain non markovian
#args = {"N":list(range(3,30,2)),"dovar":[0], "target":[1], "obsvar":[-1],  "seed":SEED, "method":["CVE", "CCVE","CCALPeps"]}
args = {"N":list(range(3,10,2)),"dovar":[0], "obsvar":[-1], "target":[1, None],  "seed":SEED, "method":["CCVE","CCALPeps"]}
outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,3)], [])

res = run_experiments(run_chain_terbin_nonmarkovian, args, outkeys)
res_file = f"{res_folder}{strdate()}_chain_terbin_nonmarkovian_ev.csv"
res.to_csv(res_file)

results2 = res

{'N': 3, 'dovar': 0, 'obsvar': -1, 'target': 1, 'seed': 1234, 'method': 'CCVE', 'time': 433.482, 'query_time': 37.699, 'lowerbound0': 0.1568627450980392, 'upperbound0': 0.6211453744493393, 'lowerbound1': 0.37885462555066085, 'upperbound1': 0.8431372549019608, 'lowerbound2': nan, 'upperbound2': nan}
{'N': 3, 'dovar': 0, 'obsvar': -1, 'target': 1, 'seed': 1234, 'method': 'CCALPeps', 'time': 807.033, 'query_time': 752.792, 'lowerbound0': 0.1570270353500937, 'upperbound0': 0.6199932673861718, 'lowerbound1': 0.3800067326138282, 'upperbound1': 0.8429729646499062, 'lowerbound2': nan, 'upperbound2': nan}
{'N': 3, 'dovar': 0, 'obsvar': -1, 'target': 1, 'seed': 2468, 'method': 'CCVE', 'time': 452.541, 'query_time': 30.447, 'lowerbound0': 0.004415011037527593, 'upperbound0': 0.004415011037527593, 'lowerbound1': 0.9955849889624724, 'upperbound1': 0.9955849889624724, 'lowerbound2': nan, 'upperbound2': nan}
{'N': 3, 'dovar': 0, 'obsvar': -1, 'target': 1, 'seed': 2468, 'method': 'CCALPeps', 'time': 8

,N,dovar,obsvar,target,seed,method,time,query_time,lowerbound0,upperbound0,lowerbound1,upperbound1,lowerbound2,upperbound2
0,3,0,-1,1,1234,CCVE,433.482,37.699,0.156863,0.621145,0.378855,0.843137,NaN,NaN
1,3,0,-1,1,1234,CCALPeps,807.033,752.792,0.157027,0.619993,0.380007,0.842973,NaN,NaN
2,3,0,-1,1,2468,CCVE,452.541,30.447,0.004415,0.004415,0.995585,0.995585,NaN,NaN
3,3,0,-1,1,2468,CCALPeps,887.833,830.696,0.004440,0.004443,0.995557,0.995560,NaN,NaN
4,3,0,-1,1,3702,CCVE,427.665,28.549,0.763780,0.763780,0.236220,0.236220,NaN,NaN
5,3,0,-1,1,3702,CCALPeps,787.294,731.913,0.762860,0.763433,0.236567,0.237140,NaN,NaN
6,3,0,-1,None,1234,CCVE,436.604,37.267,0.156863,0.621145,0.378855,0.843137,NaN,NaN
7,3,0,-1,None,1234,CCALPeps,829.073,777.459,0.157027,0.619993,0.380007,0.842973,NaN,NaN
8,3,0,-1,None,2468,CCVE,457.226,31.935,0.004415,0.004415,0.995585,0.995585,NaN,NaN
9,3,0,-1,None,2468,CCALPeps,875.444,822.152,0.004440,0.004443,0.995557,0.995560,NaN,NaN


In [36]:
results2.query("upperbound0 - lowerbound0 > 0.01")

,N,dovar,obsvar,target,seed,method,time,query_time,lowerbound0,upperbound0,lowerbound1,upperbound1,lowerbound2,upperbound2
0,3,0,-1,1,1234,CCVE,433.482,37.699,0.156863,0.621145,0.378855,0.843137,NaN,NaN
1,3,0,-1,1,1234,CCALPeps,807.033,752.792,0.157027,0.619993,0.380007,0.842973,NaN,NaN
6,3,0,-1,None,1234,CCVE,436.604,37.267,0.156863,0.621145,0.378855,0.843137,NaN,NaN
7,3,0,-1,None,1234,CCALPeps,829.073,777.459,0.157027,0.619993,0.380007,0.842973,NaN,NaN
12,5,0,-1,1,1234,CCVE,539.465,77.163,0.646288,0.941527,0.058473,0.353712,NaN,NaN
13,5,0,-1,1,1234,CCALPeps,1646.795,1590.941,0.646323,0.941197,0.058803,0.353677,NaN,NaN
18,5,0,-1,None,1234,CCVE,572.261,121.260,0.166649,0.438210,0.442772,0.660098,0.110480,0.183487
19,5,0,-1,None,1234,CCALPeps,2346.057,2292.895,0.225027,0.379131,0.451563,0.649930,0.125042,0.169306
24,7,0,-1,1,1234,CCVE,712.593,245.964,0.640000,0.940000,0.060000,0.360000,NaN,NaN
25,7,0,-1,1,1234,CCALPeps,2982.830,2925.554,0.640057,0.939667,0.060333,0.359943,NaN,NaN
